In [2]:
import torch
from brt._C.router import generate_indices_and_loads, dispatch_with_dst_indices_2d

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_dst_index=False
)
print(route_indices)
print(dst_loads)


tensor([[4, 2, 1, 0],
        [6, 0, 3, 0],
        [7, 0, 5, 0],
        [8, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([4, 1, 4, 0], device='cuda:0', dtype=torch.int32)


In [6]:
routed_gates = torch.randn((8, path_num, 2)).cuda()
print(routed_gates)
out_gates = dispatch_with_dst_indices_2d(routed_gates, route_indices, dst_loads)
print(out_gates)
print(routed_gates.transpose(0, 1))

tensor([[[ 0.6216, -0.4880],
         [-1.3436,  0.0229],
         [-0.3075, -1.6070],
         [-2.6341,  1.9152]],

        [[-0.9084, -0.1804],
         [-1.1087, -0.3289],
         [ 0.0546,  1.8197],
         [-0.5505,  2.7316]],

        [[ 0.1580,  0.8280],
         [-0.2797, -0.6031],
         [ 1.1448, -0.8272],
         [-0.0974,  1.3582]],

        [[ 0.7644, -0.3814],
         [-1.2840,  0.2500],
         [ 0.6187,  2.1708],
         [ 0.3625,  1.9453]],

        [[ 0.0802,  0.1215],
         [ 0.5953,  0.2044],
         [ 1.3774, -1.4690],
         [-0.2813,  1.4029]],

        [[-0.8915, -0.8812],
         [ 2.2945,  1.0372],
         [-0.6549,  0.0763],
         [-0.6589,  1.5777]],

        [[ 0.0827,  0.8408],
         [ 0.1506,  0.9301],
         [ 2.0818,  0.2048],
         [-1.0626, -0.6085]],

        [[ 0.0040, -0.3704],
         [-0.2571,  0.9365],
         [-1.0022, -0.5935],
         [-0.7302,  0.7561]]], device='cuda:0')
tensor([[ 0.6216, -0.4880],
        [-0